#<b><center>NLP Utah</b></center>

Importacion de librerías

In [1]:
from google.colab import drive
import pandas as pd
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from google.cloud import storage
from google.colab import auth
auth.authenticate_user()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Montando carpeta drive.

In [2]:
drive.mount('/content/drive')
%cd /content/drive/MyDrive/datalake/reviews-estados/review-Utah

Mounted at /content/drive
/content/drive/MyDrive/datalake/reviews-estados/review-Utah


In [3]:
df = pd.read_parquet('reviews.parquet')
df.info()
df.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 905248 entries, 0 to 905247
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   user_id  905248 non-null  float64
 1   gmap_id  905248 non-null  object 
 2   text     905248 non-null  object 
dtypes: float64(1), object(2)
memory usage: 20.7+ MB


,user_id,gmap_id,text
0,1.121150e+20,0x87528c3a3c5eef47:0x6de2ac4b0d5d39c2,Every time I have come into this location ever...
1,1.054794e+20,0x87528c3a3c5eef47:0x6de2ac4b0d5d39c2,Horrible service.\nI schedule my package for p...
2,1.154436e+20,0x87528c3a3c5eef47:0x6de2ac4b0d5d39c2,Mail carrier broke my front porch light. Tryin...
3,1.136965e+20,0x87528c3a3c5eef47:0x6de2ac4b0d5d39c2,Great experience here ! Friendly and helpful e...
4,1.056595e+20,0x87528c3a3c5eef47:0x6de2ac4b0d5d39c2,Me and my neighbor removed the front Fence bet...


In [4]:
textos = df['text']
textos.tail()

905243    Best dentist and crew! I love coming here and ...
905244    I was having a bad morning and running a littl...
905245    I love this dentist. They're very efficient an...
905246    They do a great job of making you feel comfort...
905247    I never thought that going to the dentist coul...
Name: text, dtype: object

<b>Tokenización</b>

Se eliminan signos de puntuación y se toma la raíz de las palabras. Se tokeniza el texto de los reviews.

In [5]:
# Initialize empty array
# to append clean text
corpus = []
for texto in textos:

    # column : "Review", row ith
    review = re.sub('[^a-zA-Z]', ' ', texto)

    # convert all cases to lower cases
    review = review.lower()

    # split to array(default delimiter is " ")
    review = review.split()

    # creating PorterStemmer object to
    # take main stem of each word
    ps = PorterStemmer()

    # loop for stemming each word
    # in string array at ith row
    review = [ps.stem(word) for word in review
                if not word in set(stopwords.words('english'))]

    # rejoin all string array elements
    # to create back into a string
    review = ' '.join(review)

    # append each string to create
    # array of clean text
    corpus.append(review)



In [6]:
df['review'] = corpus
df.drop(columns=['text'])
df.info()
df.to_parquet('reviews_token.parquet')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 905248 entries, 0 to 905247
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   user_id  905248 non-null  float64
 1   gmap_id  905248 non-null  object 
 2   text     905248 non-null  object 
 3   review   905248 non-null  object 
dtypes: float64(1), object(3)
memory usage: 27.6+ MB


Guardado en datalake de  Google Cloud:

In [7]:
project_id = 'deep-wave-407518'
bucket_name = 'yelp-and-maps-data-processed'


In [8]:
# Google Storage user.parquet
# Configura el cliente de almacenamiento
client = storage.Client(project=project_id)

# Obtén el bucket
bucket = client.get_bucket(bucket_name)

# Define la ruta del archivo Parquet en Google Cloud Storage
gcs_path = 'Maps/reviews-estados/review-Utah/reviews_token.parquet'.format(bucket_name)

# Sube el archivo Parquet a Google Cloud Storage
blob = bucket.blob(gcs_path)
blob.upload_from_filename('reviews_token.parquet')
